<a href="https://colab.research.google.com/github/leehgkor/lhg/blob/main/%EA%B5%AC%EA%B2%B0_%ED%85%8D%EC%8A%A4%ED%8A%B8_%EB%A7%88%EC%9D%B4%EB%8B%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 구결 형태소분석 말뭉치를 일반적인 형식으로 변환하는 코드

import re
import json

# 🔹 화엄.txt 파일을 업로드하거나 Google Drive에서 불러와야 함
# 예: from google.colab import files; uploaded = files.upload()

with open("/content/화엄.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

entry_pattern = re.compile(r'<[CE]?(화엄\d{2}:\d{2}(?:-\d{2})?)>\s*([^\n<]*)')
morph_pattern = re.compile(r'([^\[\]]+)\[([^\[\]]+)\]')

document = {
    "id": "화엄",
    "metadata": {},
    "sentence": []
}

for match in entry_pattern.finditer(raw_text):
    raw_id = match.group(1)
    text = match.group(2).strip()
    parts = raw_id.split(":")
    doc_id = f"화엄.{parts[0][-2:]}.{parts[1]}"

    words = []
    word_id = 1
    surface_with_gugeol = []

    if "[" in text:
        for morph_match in morph_pattern.finditer(text):
            base_form = morph_match.group(1).strip()
            tag_bundle = morph_match.group(2)
            gugeol_str = ""
            tags = [t.strip() for t in tag_bundle.split(',') if t.strip()]

            combined_gugeol = ''
            combined_pos = None
            in_dash_mode = False

            for tag in tags:
                # (1) '-' 연결된 구결: 여러 구결 → 하나의 품사
                if '-' in tag:
                    gu_char = tag[0]
                    pos = re.search(r'[A-Za-z]+', tag).group()
                    combined_gugeol += gu_char
                    combined_pos = pos
                    in_dash_mode = True
                else:
                    if in_dash_mode:
                        words.append({
                            "id": word_id,
                            "form": base_form,
                            "gugeol": combined_gugeol,
                            "pos": combined_pos
                        })
                        word_id += 1
                        gugeol_str += combined_gugeol
                        combined_gugeol = ''
                        combined_pos = None
                        in_dash_mode = False

                    # (2) '+' 복합 품사
                    if '+' in tag:
                        m = re.match(r'(?P<gugeol>[\uf000-\uf8ff])(?P<pos>.+)', tag)
                        if m:
                            gugeol = m.group("gugeol")
                            pos = m.group("pos")
                            words.append({
                                "id": word_id,
                                "form": base_form,
                                "gugeol": gugeol,
                                "pos": pos
                            })
                            word_id += 1
                            gugeol_str += gugeol

                    # (3) 일반 구결
                    else:
                        m = re.match(r'(?P<gugeol>[\uf000-\uf8ff])(?P<pos>[A-Za-z]+)', tag)
                        if m:
                            gugeol = m.group("gugeol")
                            pos = m.group("pos")
                            words.append({
                                "id": word_id,
                                "form": base_form,
                                "gugeol": gugeol,
                                "pos": pos
                            })
                            word_id += 1
                            gugeol_str += gugeol

            if in_dash_mode and combined_gugeol and combined_pos:
                words.append({
                    "id": word_id,
                    "form": base_form,
                    "gugeol": combined_gugeol,
                    "pos": combined_pos
                })
                word_id += 1
                gugeol_str += combined_gugeol

            surface_with_gugeol.append(base_form + gugeol_str)
    else:
        surface_with_gugeol = [text] if text else []

    sentence_obj = {
        "id": doc_id,
        "form": " ".join(surface_with_gugeol),
        "word": words
    }
    document["sentence"].append(sentence_obj)

# 🔹 JSON으로 저장
with open("화엄_structured.json", "w", encoding="utf-8") as f:
    json.dump(document, f, ensure_ascii=False, indent=2)

print("✅ 변환 완료: '화엄_structured_corrected.json' 저장됨")


✅ 변환 완료: '화엄_structured_corrected.json' 저장됨


In [7]:
# 형태소분석 말뭉치에서 특정 pos로 나타나는 구결자의 목록을 정리하고, 해당 구결자가 나타난 어절에서 앞뒤에 온 구결 시퀀스의 패턴 정리 및 빈도를 확인하는 코드.

# 필요한 라이브러리
import json
from collections import defaultdict
import pandas as pd

# 📁 사용자 정의 정렬 기준
custom_order = [
    "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "",
    "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "",
    "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "",
    "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", ""
]
order_map = {char: i for i, char in enumerate(custom_order)}

# 📤 업로드 도구
from google.colab import files
uploaded = files.upload()

# 🔄 JSON 파일 로딩
filename = list(uploaded.keys())[0]
with open(filename, "r", encoding="utf-8") as f:
    parsed_data = json.load(f)

sentences = parsed_data["sentence"]

# 🔍 Et 구결자 앞뒤 시퀀스 분석
et_contexts = defaultdict(lambda: defaultdict(int))

for sentence in sentences:
    words = sentence.get("word", [])
    if not words:
        continue

    current_form = None
    current_word = []

# 아래 et를 다른 pos로 바꾸면 해당 pos에 대해서도 동일한 작업 가능.
    for token in words + [{"form": None}]:  # flush last word
        if token["form"] != current_form:
            if current_word:
                for i, t in enumerate(current_word):
                    if t["pos"] == "Ec":
                        gugeol = t["gugeol"]
                        prev = ''.join(w["gugeol"] for w in current_word[:i]) or "(어절 시작)"
                        next_ = ''.join(w["gugeol"] for w in current_word[i+1:]) or "(어절 끝)"
                        et_contexts[gugeol][(prev, gugeol, next_)] += 1
            current_form = token["form"]
            current_word = []

        if token["form"] is not None:
            current_word.append(token)

# 📊 DataFrame 생성 및 정렬
rows = []
for gugeol, contexts in et_contexts.items():
    for (prev, center, next_), count in contexts.items():
        rows.append({
            "Ec_구결자": center,
            "앞_구결자_시퀀스": prev,
            "뒤_구결자_시퀀스": next_,
            "빈도": count
        })

df = pd.DataFrame(rows)
df["sort_key"] = df["Ec_구결자"].map(order_map)
df_sorted = df.sort_values(by=["sort_key", "빈도"], ascending=[True, False]).drop(columns=["sort_key"])

# 📄 텍스트 파일로 저장
lines = [
    f"[Ec: {row['Ec_구결자']}] 앞: {row['앞_구결자_시퀀스']} → 뒤: {row['뒤_구결자_시퀀스']}  ({row['빈도']}회)"
    for _, row in df_sorted.iterrows()
]

output_filename = "유가_Ec_구결자_정렬됨_어절단위_앞뒤시퀀스.txt"
with open(output_filename, "w", encoding="utf-8") as f:
    f.write("\n".join(lines))

# 📥 다운로드 링크
files.download(output_filename)


Saving 유가_structured.json to 유가_structured.json


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [9]:
# <화엄>에만 나타나는 패턴의 확인 [*]로 표시

# 필요한 라이브러리
import re
import unicodedata
from google.colab import files

# 📤 파일 업로드 (다섯 문헌 텍스트 업로드 필요)
uploaded = files.upload()

# 📁 파일 이름 지정
files_by_name = {
    "화엄": "화엄_Ec_구결자_정렬됨_어절단위_앞뒤시퀀스.txt",
    "화소": "화소_Ec_구결자_정렬됨_어절단위_앞뒤시퀀스.txt",
    "유가": "유가_Ec_구결자_정렬됨_어절단위_앞뒤시퀀스.txt",
    "금광": "금광_Ec_구결자_정렬됨_어절단위_앞뒤시퀀스.txt",
    "구인": "구인_Ec_구결자_정렬됨_어절단위_앞뒤시퀀스.txt",
}

# 🔍 유틸 함수: 빈도 정보 제거 + 정규화 + 공백 제거
def remove_frequency_info(line: str) -> str:
    return re.sub(r"\s*\(\d+회\)", "", line).strip()

def normalize_line(line: str) -> str:
    return unicodedata.normalize("NFKC", line).replace(" ", "").strip()

# 🔄 각 파일에서 비교용 시퀀스 추출
def extract_raw_lines(filepath):
    lines = set()
    with open(filepath, "r", encoding="utf-8") as f:
        for line in f:
            if "앞:" in line and "→ 뒤:" in line:
                lines.add(line.strip())
    return lines

# 📚 모든 문헌에서 비교용 시퀀스 정리
norm_nofreq_lines_by_text = {
    name: set(normalize_line(remove_frequency_info(line)) for line in extract_raw_lines(fname))
    for name, fname in files_by_name.items()
}

# 🔁 비교 대상: 화엄 이외 4문헌 통합
norm_nofreq_other_lines = (
    norm_nofreq_lines_by_text["화소"]
    | norm_nofreq_lines_by_text["유가"]
    | norm_nofreq_lines_by_text["금광"]
    | norm_nofreq_lines_by_text["구인"]
)

# 🔧 화엄 원본 라인 다시 불러오기
with open(files_by_name["화엄"], "r", encoding="utf-8") as f:
    hwaeom_lines = f.readlines()

# ⭐ [*] 표식 부여
final_lines = []
for line in hwaeom_lines:
    stripped_line = remove_frequency_info(line)
    norm_line = normalize_line(stripped_line)
    if norm_line.startswith("[Ec:") and "앞:" in norm_line and "→뒤:" in norm_line:
        if norm_line not in norm_nofreq_other_lines:
            final_lines.append(line.rstrip() + "  [*]\n")
        else:
            final_lines.append(line)
    else:
        final_lines.append(line)

# 💾 결과 저장 및 다운로드
output_path = "화엄_Ec_고유시퀀스_표시됨.txt"
with open(output_path, "w", encoding="utf-8") as f:
    f.writelines(final_lines)

files.download(output_path)


Saving 구인_Ec_구결자_정렬됨_어절단위_앞뒤시퀀스.txt to 구인_Ec_구결자_정렬됨_어절단위_앞뒤시퀀스 (1).txt
Saving 금광_Ec_구결자_정렬됨_어절단위_앞뒤시퀀스.txt to 금광_Ec_구결자_정렬됨_어절단위_앞뒤시퀀스 (1).txt
Saving 유가_Ec_구결자_정렬됨_어절단위_앞뒤시퀀스.txt to 유가_Ec_구결자_정렬됨_어절단위_앞뒤시퀀스 (1).txt
Saving 화소_Ec_구결자_정렬됨_어절단위_앞뒤시퀀스.txt to 화소_Ec_구결자_정렬됨_어절단위_앞뒤시퀀스 (1).txt
Saving 화엄_Ec_구결자_정렬됨_어절단위_앞뒤시퀀스.txt to 화엄_Ec_구결자_정렬됨_어절단위_앞뒤시퀀스 (1).txt


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [10]:
# 화엄에 나타나지 않는 패턴의 확인

# 📦 필요한 라이브러리
import re
import unicodedata
from google.colab import files

# 📤 파일 업로드 (다섯 개 파일 업로드)
uploaded = files.upload()

# 📁 파일 이름 매핑
files_by_name = {
    "화엄": "화엄_Ec_구결자_정렬됨_어절단위_앞뒤시퀀스.txt",
    "화소": "화소_Ec_구결자_정렬됨_어절단위_앞뒤시퀀스.txt",
    "유가": "유가_Ec_구결자_정렬됨_어절단위_앞뒤시퀀스.txt",
    "금광": "금광_Ec_구결자_정렬됨_어절단위_앞뒤시퀀스.txt",
    "구인": "구인_Ec_구결자_정렬됨_어절단위_앞뒤시퀀스.txt",
}

# 🔧 정규화 + 빈도 제거 함수 정의
def remove_frequency_info(line: str) -> str:
    return re.sub(r"\s*\(\d+회\)", "", line).strip()

def normalize_line(line: str) -> str:
    return unicodedata.normalize("NFKC", line).replace(" ", "").strip()

def extract_raw_lines(filepath):
    lines = set()
    with open(filepath, "r", encoding="utf-8") as f:
        for line in f:
            if "앞:" in line and "→ 뒤:" in line:
                lines.add(line.strip())
    return lines

# 🔍 화엄 기준 시퀀스 정리
hwaeom_norm_set = set(
    normalize_line(remove_frequency_info(line))
    for line in extract_raw_lines(files_by_name["화엄"])
)

# 📊 다른 문헌에서 화엄에 없는 시퀀스를 수집하고 출처 병기
unseen_with_source = {}

for name in ["유가", "화소", "금광", "구인"]:
    with open(files_by_name[name], "r", encoding="utf-8") as f:
        for line in f:
            cleaned_line = remove_frequency_info(line)
            norm_line = normalize_line(cleaned_line)
            if norm_line.startswith("[Ec:") and "앞:" in norm_line and "→뒤:" in norm_line:
                if norm_line not in hwaeom_norm_set:
                    if cleaned_line not in unseen_with_source:
                        unseen_with_source[cleaned_line] = set()
                    unseen_with_source[cleaned_line].add(name)

# 📄 정렬 및 저장
lines_with_sources = [
    f"{line}  [{'/'.join(sorted(sources))}]"
    for line, sources in unseen_with_source.items()
]
lines_with_sources_sorted = sorted(lines_with_sources)

output_path = "화엄에_없는_Ec_시퀀스_다른문헌중_출처표시.txt"
with open(output_path, "w", encoding="utf-8") as f:
    for line in lines_with_sources_sorted:
        f.write(line + "\n")

# 📥 다운로드
files.download(output_path)


Saving 구인_Ec_구결자_정렬됨_어절단위_앞뒤시퀀스.txt to 구인_Ec_구결자_정렬됨_어절단위_앞뒤시퀀스 (2).txt
Saving 금광_Ec_구결자_정렬됨_어절단위_앞뒤시퀀스.txt to 금광_Ec_구결자_정렬됨_어절단위_앞뒤시퀀스 (2).txt
Saving 유가_Ec_구결자_정렬됨_어절단위_앞뒤시퀀스.txt to 유가_Ec_구결자_정렬됨_어절단위_앞뒤시퀀스 (2).txt
Saving 화소_Ec_구결자_정렬됨_어절단위_앞뒤시퀀스.txt to 화소_Ec_구결자_정렬됨_어절단위_앞뒤시퀀스 (2).txt
Saving 화엄_Ec_구결자_정렬됨_어절단위_앞뒤시퀀스.txt to 화엄_Ec_구결자_정렬됨_어절단위_앞뒤시퀀스 (2).txt


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>